This notebook calculates haversine distances for all stamp points. This matrix is saved and will be used for future optimization 

In [60]:
import pandas as pd #type: ignore
import os
import sys
import pathlib
import numpy as np #type: ignore
path_to_harz = pathlib.Path(
    r"C:\Users\oleh.bakumenko\PythonWorkspace\PycharmProjects\sandbox\harz")
all_stamps = pd.read_excel(os.path.join(
    path_to_harz, 'excel', 'HWN2024_export_gpx.xlsx'))

In [61]:
num_elements = len(all_stamps["number"])

distance_matrix = np.zeros((num_elements, num_elements))
distance_matrix, distance_matrix.shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 (222, 222))

In [62]:
import math


def get_stamp_from_num(number):
    return all_stamps[all_stamps["number"] == number]


def get_coord_from_stamp(a):
    return np.array((a["latitude"], a["longitude"])).flatten()


def calculate_haversine_distance(coord_a, coord_b):
    """
    taken from here 
    https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

    Returns: 
        haversine distance between 2 coordinates in meters
    """
    lat1, lon1, = coord_a[0], coord_a[1]
    lat2, lon2, = coord_b[0], coord_b[1]
    r = 6371000  # in meters
    p = math.pi / 180

    a = 0.5 - math.cos((lat2-lat1)*p)/2 + math.cos(lat1*p) * \
        math.cos(lat2*p) * (1-math.cos((lon2-lon1)*p))/2
    return np.round(2 * r * math.asin(math.sqrt(a)), 2)

# TODO later add the real distance calculation by sending the coordinates to the komoot or OpenStreetMaps

In [63]:
# test the functions
a = get_stamp_from_num(1)
b = get_stamp_from_num(10)
coord_a = get_coord_from_stamp(a)
coord_b = get_coord_from_stamp(b)
calculate_haversine_distance(coord_a, coord_b)

np.float64(6146.8)

In [64]:
for row in range(num_elements):
    for col in range(num_elements):
        stamp_a = get_stamp_from_num(row+1)
        stamp_b = get_stamp_from_num(col+1)
        coord_a = get_coord_from_stamp(stamp_a)
        coord_b = get_coord_from_stamp(stamp_b)
        distance_matrix[row, col] = calculate_haversine_distance(
            coord_a, coord_b)

In [65]:
np.save(os.path.join(path_to_harz, 'arrays',
        'all_stamps_distances.npy'), distance_matrix)

In [66]:
np.unravel_index(distance_matrix.argmax(), distance_matrix.shape)

(np.int64(101), np.int64(207))

In [67]:
distance_matrix[101, 207]

np.float64(84553.01)

In [68]:
# whereever the distance is 0, set it to infinity
x = distance_matrix
x[x == 0] = np.inf
np.unravel_index(x.argmin(), x.shape)

(np.int64(65), np.int64(68))

In [69]:
distance_matrix[65, 68]

np.float64(301.46)

: 